In [1]:
import pandas as pd
import numpy as np
import math

In [56]:
from datetime import date, timedelta
import time
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sns
%config InlineBackend.figure_format = 'svg'
%matplotlib inline 

In [3]:
# mta_data = pd.read_csv("/media/hdd1/merged_file.txt")

In [4]:
# mta_data["DATE TIME"] = mta_data["DATE"] + " " + mta_data["TIME"]

In [9]:
# mta_data["DATE TIME"].head()
# mta_data["DATETIME2"] = pd.to_datetime(mta_data["DATE TIME"])

In [2]:
cd "/Users/Tahmid/Documents/Metis/Data Science/projects/project1/Pickle Files"

/Users/Tahmid/Documents/Metis/Data Science/projects/project1/Pickle Files


In [3]:
ls

mta/              mta.tar.gz        mta_datetime.pkl


In [2]:
import pickle
mta_data =  pd.read_pickle("/Users/Tahmid/Documents/Metis/Data Science/projects/project1/Pickle Files/mta/mta_datetime.pkl")

In [3]:
# Drop outlier EXIT and ENTRANCE times that are zero
mta_cleaned_data = mta_data[(mta_data.EXITS != 0) | (mta_data.ENTRIES != 0)]

In [4]:
len(mta_cleaned_data)

10344462

In [5]:
mta_cleaned_data.to_pickle("mta_datetime.pkl")

In [6]:
#Lets Clean the Data
#Step 1 we are going to remove all the bad dates data which are not actual 4 hour window
# There are datasets which are poorly aggregated
valid_time_list = []
for i in mta_cleaned_data["TIME"].unique():
    if (i.split(":")[1] == "00" and i.split(":")[2] =="00"):
        valid_time_list.append(i)

mta_cleaned_data_filtered_valida_date = mta_cleaned_data.loc[mta_cleaned_data["TIME"].isin(valid_time_list)]       

In [7]:
# Confirming that we've gotten rid of the non-zero times
mta_cleaned_data_filtered_valida_date.TIME.unique()

array(['03:00:00', '07:00:00', '11:00:00', '15:00:00', '19:00:00',
       '23:00:00', '01:00:00', '05:00:00', '09:00:00', '13:00:00',
       '17:00:00', '21:00:00', '00:00:00', '04:00:00', '08:00:00',
       '12:00:00', '16:00:00', '20:00:00', '02:00:00', '06:00:00',
       '10:00:00', '14:00:00', '18:00:00', '22:00:00'], dtype=object)

In [8]:
#Now lets check the rows of the files
# number of rows before was 10344462
len(mta_cleaned_data_filtered_valida_date)

9439309

In [9]:
# So total data that we cleaned is 9439309
# So the percentage change
change_percent = ((len(mta_cleaned_data) - len(mta_cleaned_data_filtered_valida_date)))
print(change_percent/len(mta_cleaned_data)*100)

8.750121562629356


In [28]:
# Percentatge of files dropped in 8.75%, which is insignificant

In [30]:
# DELETE THIS KERNEL LATER

# Lets add the total for all the Entries and Exits
#This will give us a gross idea of the top 20 stations
#The turnstile data always increment. So we would have to do a diff between the rows
# We are going to create a total column and add the entries and exits
mta_cleaned_data["TOTAL"] = mta_cleaned_data["ENTRIES"] + mta_cleaned_data["EXITS"]

/home/sdutta/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
mta_cleaned_data_filtered_valida_date["TOTAL"] = mta_cleaned_data_filtered_valida_date["ENTRIES"] + mta_cleaned_data_filtered_valida_date["EXITS"]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
# Confirming "TOTAL" column has been added
mta_cleaned_data_filtered_valida_date.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2,TOTAL
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,03:00:00,REGULAR,6463109,2187558,12/30/2017 03:00:00,12/30/2017 03:00:00,2017-12-30 03:00:00,8650667
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,07:00:00,REGULAR,6463119,2187567,12/30/2017 07:00:00,12/30/2017 07:00:00,2017-12-30 07:00:00,8650686
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,11:00:00,REGULAR,6463178,2187654,12/30/2017 11:00:00,12/30/2017 11:00:00,2017-12-30 11:00:00,8650832
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,15:00:00,REGULAR,6463373,2187736,12/30/2017 15:00:00,12/30/2017 15:00:00,2017-12-30 15:00:00,8651109
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,19:00:00,REGULAR,6463712,2187803,12/30/2017 19:00:00,12/30/2017 19:00:00,2017-12-30 19:00:00,8651515


In [12]:
# Group by station and sort by Total to determine 20 stations with highest traffic
mta_cleaned_data_filtered_valida_date.groupby(["STATION"])[["TOTAL"]].sum().sort_values("TOTAL", ascending=False).reset_index().head(20)

,STATION,TOTAL
0,42 ST-PORT AUTH,30073275509855
1,23 ST,28015047978750
2,34 ST-HERALD SQ,26653914721598
3,CANAL ST,25427442946053
4,TIMES SQ-42 ST,22662320895477
5,CHAMBERS ST,20752064050159
6,FULTON ST,18995465726594
7,104 ST,18208421962867
8,72 ST,17978959372705
9,14 ST,17069322993663


In [13]:
mta_20_totals = mta_cleaned_data_filtered_valida_date.groupby(["STATION"])[["TOTAL"]].sum().sort_values("TOTAL", ascending=False).reset_index().head(20)

In [14]:
# Capturing names of top 20 stations as a list to use narrow down our dataframe
mta_20_stations = list(mta_20_totals.STATION)
mta_20_stations

['42 ST-PORT AUTH',
 '23 ST',
 '34 ST-HERALD SQ',
 'CANAL ST',
 'TIMES SQ-42 ST',
 'CHAMBERS ST',
 'FULTON ST',
 '104 ST',
 '72 ST',
 '14 ST',
 '125 ST',
 '34 ST-PENN STA',
 '59 ST',
 'HIGH ST',
 '59 ST COLUMBUS',
 '3 AV-149 ST',
 'LEXINGTON AV/53',
 '57 ST-7 AV',
 '183 ST',
 'DEKALB AV']

In [15]:
# Create dataframe with data for the top 20 desired stations
mta_20_data_cum = mta_cleaned_data_filtered_valida_date.loc[mta_cleaned_data_filtered_valida_date["STATION"].isin(mta_20_stations)]

In [16]:
# Create dataframe in which the data is sorted appropriately in order to do the diff
mta_20_data_dif = mta_20_data_cum.sort_values(['STATION','C/A','UNIT','SCP','DATETIME2'], ascending=True)

In [17]:
# Create new ENTRIES column with actuals
mta_20_data_dif["ENTRIES_A"] = mta_20_data_dif["ENTRIES"].diff()

In [18]:
# Create new EXITS column with actuals
mta_20_data_dif["EXITS_A"] = mta_20_data_dif["EXITS"].diff()

In [19]:
# Create new ENTRIES column with actuals
mta_20_data_dif["TOTAL_A"] = mta_20_data_dif["TOTAL"].diff()

In [20]:
# Drop the first row using it's index, which has NaN values due to the .diff 
mta_20_data_dif.drop(39635, inplace=True)

In [21]:
mta_20_data_dif.head(100)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2,TOTAL,ENTRIES_A,EXITS_A,TOTAL_A
39636,J034,R007,00-00-00,104 ST,JZ,BMT,12/30/2017,07:00:00,REGULAR,5831706,4798280,12/30/2017 07:00:00,NaN,2017-12-30 07:00:00,10629986,54.0,6.0,60.0
39637,J034,R007,00-00-00,104 ST,JZ,BMT,12/30/2017,11:00:00,REGULAR,5831837,4798293,12/30/2017 11:00:00,NaN,2017-12-30 11:00:00,10630130,131.0,13.0,144.0
39638,J034,R007,00-00-00,104 ST,JZ,BMT,12/30/2017,15:00:00,REGULAR,5831928,4798310,12/30/2017 15:00:00,NaN,2017-12-30 15:00:00,10630238,91.0,17.0,108.0
39639,J034,R007,00-00-00,104 ST,JZ,BMT,12/30/2017,19:00:00,REGULAR,5832005,4798338,12/30/2017 19:00:00,NaN,2017-12-30 19:00:00,10630343,77.0,28.0,105.0
39640,J034,R007,00-00-00,104 ST,JZ,BMT,12/30/2017,23:00:00,REGULAR,5832052,4798351,12/30/2017 23:00:00,NaN,2017-12-30 23:00:00,10630403,47.0,13.0,60.0
39641,J034,R007,00-00-00,104 ST,JZ,BMT,12/31/2017,03:00:00,REGULAR,5832056,4798365,12/31/2017 03:00:00,NaN,2017-12-31 03:00:00,10630421,4.0,14.0,18.0
39642,J034,R007,00-00-00,104 ST,JZ,BMT,12/31/2017,07:00:00,REGULAR,5832093,4798371,12/31/2017 07:00:00,NaN,2017-12-31 07:00:00,10630464,37.0,6.0,43.0
39643,J034,R007,00-00-00,104 ST,JZ,BMT,12/31/2017,11:00:00,REGULAR,5832179,4798373,12/31/2017 11:00:00,NaN,2017-12-31 11:00:00,10630552,86.0,2.0,88.0
39644,J034,R007,00-00-00,104 ST,JZ,BMT,12/31/2017,15:00:00,REGULAR,5832251,4798387,12/31/2017 15:00:00,NaN,2017-12-31 15:00:00,10630638,72.0,14.0,86.0
39645,J034,R007,00-00-00,104 ST,JZ,BMT,12/31/2017,19:00:00,REGULAR,5832311,4798399,12/31/2017 19:00:00,NaN,2017-12-31 19:00:00,10630710,60.0,12.0,72.0


In [22]:
# Check to see if there are outlier (there are)
mta_20_data_dif.describe()

,ENTRIES,EXITS,TOTAL,ENTRIES_A,EXITS_A,TOTAL_A
count,1.813926e+06,1.813926e+06,1.813926e+06,1.813926e+06,1.813926e+06,1.813926e+06
mean,1.065387e+08,9.402106e+07,2.005597e+08,1.088126e+00,-1.261224e+00,-1.730980e-01
std,3.650975e+08,3.353910e+08,6.692370e+08,1.229660e+07,1.081458e+07,2.201280e+07
min,0.000000e+00,0.000000e+00,1.000000e+00,-2.117211e+09,-2.050113e+09,-3.817479e+09
25%,1.179677e+06,6.690498e+05,2.468727e+06,1.400000e+01,1.300000e+01,3.700000e+01
50%,4.462862e+06,2.904554e+06,8.372294e+06,9.700000e+01,7.600000e+01,2.190000e+02
75%,1.046619e+07,8.289220e+06,1.937875e+07,2.990000e+02,2.250000e+02,5.830000e+02
max,2.130153e+09,2.050239e+09,3.828507e+09,2.121981e+09,2.035992e+09,3.803617e+09


In [27]:
# Limit data to the one year
mta_20_data_year = mta_20_data_dif[(mta_20_data_dif["DATETIME2"] > "2018-03-01")]

In [34]:
mta_20_data_year.describe()

,ENTRIES,EXITS,TOTAL,ENTRIES_A,EXITS_A,TOTAL_A
count,1.508342e+06,1.508342e+06,1.508342e+06,1.508342e+06,1.508342e+06,1.508342e+06
mean,1.067616e+08,9.412575e+07,2.008874e+08,-2.866140e+03,-1.998733e+03,-4.864874e+03
std,3.652319e+08,3.352749e+08,6.690575e+08,6.820273e+06,5.305369e+06,1.168792e+07
min,0.000000e+00,0.000000e+00,1.000000e+00,-2.113960e+09,-2.050113e+09,-3.603884e+09
25%,1.176337e+06,6.582582e+05,2.441811e+06,1.500000e+01,1.300000e+01,3.700000e+01
50%,4.450550e+06,2.916106e+06,8.374985e+06,9.900000e+01,7.800000e+01,2.250000e+02
75%,1.050887e+07,8.296449e+06,1.933039e+07,3.040000e+02,2.290000e+02,5.920000e+02
max,2.130060e+09,2.050239e+09,3.828507e+09,2.105700e+09,2.035992e+09,3.263848e+09


In [51]:
mta_20_data_year_out = mta_20_data_year[(mta_20_data_year["ENTRIES_A"] > float(0)) | (mta_20_data_year["EXITS_A"] > float(0)) | (mta_20_data_year["TOTAL_A"] > float(0))]

In [52]:
mta_20_data_year_out.describe()

,ENTRIES,EXITS,TOTAL,ENTRIES_A,EXITS_A,TOTAL_A
count,1.353051e+06,1.353051e+06,1.353051e+06,1.353051e+06,1.353051e+06,1.353051e+06
mean,1.015738e+08,8.992042e+07,1.914942e+08,1.628707e+04,1.229622e+04,2.858328e+04
std,3.517501e+08,3.298357e+08,6.561095e+08,4.742256e+06,3.694070e+06,8.058473e+06
min,0.000000e+00,0.000000e+00,2.000000e+00,-5.686752e+07,-4.420210e+07,-4.808436e+07
25%,1.569080e+06,1.017008e+06,3.428941e+06,3.000000e+01,2.600000e+01,7.700000e+01
50%,4.864411e+06,3.178635e+06,8.979170e+06,1.300000e+02,1.000000e+02,2.840000e+02
75%,1.058242e+07,8.608053e+06,1.948291e+07,3.380000e+02,2.560000e+02,6.450000e+02
max,2.115781e+09,2.050239e+09,3.828507e+09,2.105700e+09,2.035992e+09,3.263848e+09


In [ ]:
# df = pd.DataFrame(np.random.randn(100, 3))

# from scipy import stats
# df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [ ]:
# mta_20_data_dif[(np.abs(stats.zscore(mta_20_data_dif))<3).TOTAL_A(axis=1)]

In [58]:
mta_test = mta_20_data_year_out.sample